In [ ]:
!pip install neuralforecast

In [ ]:
import pandas as pd
import numpy as np
# python -m pip install yahoo-finance; pip install yahoo-finance; pip install yfinance --upgrade --no-cache-dir
# import yfinance as yf
# pip install --upgrade ta; pip install ta
# import ta as ta
import sklearn as sk
from sklearn import preprocessing
from scipy.stats import t
import tensorflow as tf
from datetime import date, datetime, timedelta
# from arch import arch_model
import matplotlib.pyplot as plt
import time
from tqdm import tqdm

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss, MSE, MAE
from neuralforecast.tsdataset import TimeSeriesDataset, TimeSeriesLoader
from numpy.random import seed
from random import randrange
from neuralforecast.losses.numpy import rmse, mape
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic

In [ ]:
from neuralforecast.models import NBEATS

In [ ]:
!pip install yahoo-finance

In [ ]:
import yfinance as yf

In [ ]:
nifty = yf.download("^CRSLDX",start='2012-01-01', end='2024-01-25', progress=False)
nifty

In [ ]:
nifty.index = pd.to_datetime(nifty.index)

In [ ]:
nifty = nifty.resample('D').fillna(method='ffill')

In [ ]:
nifty

In [ ]:
# Ensure the 'Price' column is numeric
# test_df['Price'] = pd.to_numeric(test_df['Price'], errors='coerce')

# If there are any None values after conversion, this will remove those rows
# df.dropna(subset=['Price'], inplace=True)

window = 21
trd_days = 252
ann_factor = window / trd_days

# Calculate log returns
nifty['log_return'] = np.log(nifty['Close'] / nifty['Close'].shift(1))

# Calculate realized variance
rlz_var = nifty['log_return'].rolling(window=window).var() * ann_factor

# Calculate realized volatility
rlz_vol = np.sqrt(rlz_var)

# You might want to store the realized volatility in the DataFrame
nifty['realized_volatility'] = rlz_vol

In [ ]:
nifty = nifty.reset_index(drop=False)
nifty

In [ ]:
nifty = nifty.dropna()
nifty

In [ ]:
nifty['unique_id'] = '1'
nifty.rename(columns={'Date':'ds'}, inplace=True)
nifty.rename(columns={'realized_volatility':'y'}, inplace=True)

In [ ]:
train = nifty.loc[:'2017-01-01'].reset_index(drop=False)
# val = df.loc['2022-11-30':'2022-12-30'].reset_index(drop=False)
test = nifty.loc['2017-01-01':].reset_index(drop=False)

In [ ]:
model = NBEATS(h=len(test), input_size=1,
                loss=MQLoss(level=[90]),
                #  loss = DistributionLoss(distribution='Poisson', level=[90])
                # scaler_type='minmax',
                #  scaler_type='standard',
                learning_rate= 0.001,
                stack_types=['identity', 'identity'],#,'trend', 'seasonality'
                n_blocks=[1,1],
                mlp_units= [[512, 512], [512, 512]],
                # windows_batch_size=60,
                num_lr_decays=5,
                # val_check_steps=20,
                n_harmonics=0, n_polynomials=0,

                max_steps=200,
                # early_stop_patience_steps=2,

                )

In [ ]:
fcst = NeuralForecast(
    models=[model],
    freq='D'
  )

In [ ]:
fcst.fit(df=train)

In [ ]:
forecasts = fcst.predict()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import make_scorer

#metric used to measure the model (Root mean squared error)
def rmse1(y_actual, y_pred):
    rmse = np.sqrt(mean_squared_error(y_actual,y_pred))
    return rmse
rmse_cv = make_scorer(rmse1, greater_is_better=False)

def mse1(y_actual, y_pred):
    mse = mean_squared_error(y_actual,y_pred)
    return mse
mse_cv = make_scorer(mse1, greater_is_better=False)

def mae1(y_actual, y_pred):
    mae = mean_absolute_error(y_actual,y_pred)
    return mae
mae_cv = make_scorer(mae1, greater_is_better=False)

def smape1(a, f):
    return 1/len(a) * np.sum(2*np.abs(f-a)/(np.abs(a)+np.abs(f))*100)


In [ ]:
forecast_rmse = rmse1(test['y'], forecasts['NBEATS-median'])
print(forecast_rmse)
forecast_mse = mse1(test['y'], forecasts['NBEATS-median'])
print(forecast_mse)
forecast_mae = mae1(test['y'], forecasts['NBEATS-median'])
print(forecast_mae)